In [ ]:
!pip install vosk
!pip install librosa
!pip install torchaudio
!pip install torchvision
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import os
def extract_audio(video_path, audio_path):
    # Run ffmpeg command to extract audio
    os.system(f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 1 {audio_path}")

# Define paths to your video folders
folder1_path = "/content/hate_videos"
folder2_path = "/content/non_hate_videos"

# Define output folder paths for audio files
output_folder1_path = "/content/output_hate"
output_folder2_path = "/content/output_non_hate"

# Create output folders if they don't exist
os.makedirs(output_folder1_path, exist_ok=True)
os.makedirs(output_folder2_path, exist_ok=True)

# Iterate over files in folder 1
for filename in os.listdir(folder1_path):
    if filename.endswith(".mp4"):
        video_path = os.path.join(folder1_path, filename)
        audio_path = os.path.join(output_folder1_path, filename.replace(".mp4", ".wav"))
        extract_audio(video_path, audio_path)

# Iterate over files in folder 2
for filename in os.listdir(folder2_path):
    if filename.endswith(".mp4"):
        video_path = os.path.join(folder2_path, filename)
        audio_path = os.path.join(output_folder2_path, filename.replace(".mp4", ".wav"))
        extract_audio(video_path, audio_path)


In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip vosk-model-small-en-us-0.15.zip -d vosk-model


--2024-04-24 09:47:48--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip.2’

vosk-model-small-en 100%[===================>]  39.30M  19.4MB/s    in 2.0s    

2024-04-24 09:47:51 (19.4 MB/s) - ‘vosk-model-small-en-us-0.15.zip.2’ saved [41205931/41205931]

Archive:  vosk-model-small-en-us-0.15.zip
replace vosk-model/vosk-model-small-en-us-0.15/am/final.mdl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import wave
import vosk

# Initialize Vosk model
model_path = "/content/vosk-model/vosk-model-small-en-us-0.15"
vosk.SetLogLevel(-1)
model = vosk.Model(model_path)

# Function to transcribe audio
def transcribe_audio(audio_path):
    wf = wave.open(audio_path, "rb")
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        print("Audio file must be WAV format mono PCM.")
        exit(1)
    rec = vosk.KaldiRecognizer(model, wf.getframerate())
    transcribed_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = rec.Result()
            transcribed_text += result
    result = rec.FinalResult()
    transcribed_text += result
    return transcribed_text

# Define paths to your audio folders
folder1_path = "/content/output_hate"
folder2_path = "/content/output_non_hate"

# Transcribe audio files in folder 1
transcribed_texts_folder1 = []
for filename in os.listdir(folder1_path):
    if filename.endswith(".wav"):
        audio_path = os.path.join(folder1_path, filename)
        transcribed_text = transcribe_audio(audio_path)
        transcribed_texts_folder1.append(transcribed_text)

# Transcribe audio files in folder 2
transcribed_texts_folder2 = []
for filename in os.listdir(folder2_path):
    if filename.endswith(".wav"):
        audio_path = os.path.join(folder2_path, filename)
        transcribed_text = transcribe_audio(audio_path)
        transcribed_texts_folder2.append(transcribed_text)

# Now you have transcribed texts for each folder
# You can pass these transcribed texts to your BERT model for feature extraction


In [ ]:
!pip install transformers


In [ ]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to extract BERT features
def extract_bert_features(texts):
    # Tokenize input texts
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')
    # Forward pass through BERT model
    outputs = model(tokenized_texts)
    # Extract features from BERT output
    pooled_output = outputs.pooler_output
    return pooled_output

# Example usage:
# Extract BERT features for transcribed texts in folder 1
bert_features_folder1 = extract_bert_features(transcribed_texts_folder1)

# Extract BERT features for transcribed texts in folder 2
bert_features_folder2 = extract_bert_features(transcribed_texts_folder2)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [ ]:
bert_features_folder1.shape


TensorShape([20, 768])

In [ ]:
bert_features_folder2.shape

TensorShape([20, 768])

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

# Function to extract frames from videos
def extract_frames(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for _ in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB format
        frames.append(frame)
    cap.release()
    return frames

# Function to extract features from frames using ViT
def extract_vit_features(frames):
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    preprocessed_frames = [preprocess_input(cv2.resize(frame, (224, 224))) for frame in frames]
    features = model.predict(np.array(preprocessed_frames))
    return features

# Define paths to your video folders
hate_videos_path = "/content/hate_videos"
non_hate_videos_path = "/content/non_hate_videos"

# Extract features for hate videos
hate_video_features = []
for filename in os.listdir(hate_videos_path):
    if filename.endswith(".mp4"):
        video_path = os.path.join(hate_videos_path, filename)
        frames = extract_frames(video_path)
        features = extract_vit_features(frames)
        hate_video_features.append(features)

# Extract features for non-hate videos
non_hate_video_features = []
for filename in os.listdir(non_hate_videos_path):
    if filename.endswith(".mp4"):
        video_path = os.path.join(non_hate_videos_path, filename)
        frames = extract_frames(video_path)
        features = extract_vit_features(frames)
        non_hate_video_features.append(features)

# Now you have extracted ViT features for frames from all hate and non-hate videos


4/4 [==============================] - 59s 13s/step


In [ ]:
import numpy as np
import os
import librosa

# Function to extract MFCC features from audio file
def extract_mfcc(audio_path, num_mfcc=40, max_mfcc_length=500):
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfcc)
    # Pad or truncate MFCC features to the fixed length
    if mfccs.shape[1] >= max_mfcc_length:
        mfccs = mfccs[:, :max_mfcc_length]
    else:
        mfccs = np.pad(mfccs, ((0, 0), (0, max_mfcc_length - mfccs.shape[1])), mode='constant', constant_values=0)
    # Normalize MFCCs
    mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)
    return mfccs

# Define paths to your audio folders
folder1_path = "/content/output_hate"
folder2_path = "/content/output_non_hate"

max_mfcc_length = 790

# Extract MFCC features for audio files in folder 1 (hate)
mfcc_features_folder1 = []
for filename in os.listdir(folder1_path):
    if filename.endswith(".wav"):
        audio_path = os.path.join(folder1_path, filename)
        mfccs = extract_mfcc(audio_path, max_mfcc_length=max_mfcc_length)
        mfcc_features_folder1.append(mfccs)

# Extract MFCC features for audio files in folder 2 (non-hate)
mfcc_features_folder2 = []
for filename in os.listdir(folder2_path):
    if filename.endswith(".wav"):
        audio_path = os.path.join(folder2_path, filename)
        mfccs = extract_mfcc(audio_path, max_mfcc_length=max_mfcc_length)
        mfcc_features_folder2.append(mfccs)


In [ ]:
# Assuming mfcc_features_folder1 is a list of MFCC features
mfcc_lengths = [len(mfcc) for mfcc in mfcc_features_folder2]

# Check if all lengths are the same
if len(set(mfcc_lengths)) == 1:
    print("All MFCC features have the same length.")
else:
    print("MFCC features have varying lengths.")


All MFCC features have the same length.


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
bert_features_folder2_np = np.array(bert_features_folder2)
bert_features_folder1_np = np.array(bert_features_folder1)

mfcc_features_folder1_np = np.array(mfcc_features_folder1)
mfcc_features_folder2_np = np.array(mfcc_features_folder2)
hate_video_features_np = np.array(hate_video_features)
non_hate_video_features_np = np.array(non_hate_video_features)
# num_samples = len(hate_video_features)
# num_mfcc_features = len(mfcc_features_folder1[0])  # Assuming all MFCC features have the same length
# mfcc_features_folder1_reshaped = mfcc_features_folder1_np.reshape(num_samples, num_mfcc_features)
# mfcc_features_folder2_reshaped = mfcc_features_folder2_np.reshape(num_samples, num_mfcc_features)


In [ ]:
print(hate_video_features_np.shape)
print(mfcc_features_folder2_np.shape)
print(bert_features_folder2.shape)

(20, 100, 7, 7, 512)
(20, 40, 790)
(20, 768)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define fusion model architecture
class FusionModel(Model):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.bert_dense = layers.Dense(256, activation='relu')
        self.mfcc_dense = layers.Dense(256, activation='relu')
        self.mfcc_flatten = layers.Flatten()  # Add flatten layer for MFCC output
        self.video_conv = layers.Conv3D(256, (3, 3, 3), activation='relu')
        self.flatten = layers.Flatten()
        self.fusion_dense = layers.Dense(512, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        bert_features, mfcc_features, video_features = inputs
        bert_output = self.bert_dense(bert_features)
        mfcc_output = self.mfcc_dense(mfcc_features)
        mfcc_output = self.mfcc_flatten(mfcc_output)  # Flatten MFCC output
        video_output = self.video_conv(video_features)
        video_output = self.flatten(video_output)
        combined_features = tf.concat([bert_output, mfcc_output, video_output], axis=-1)
        fusion_output = self.fusion_dense(combined_features)
        output = self.output_layer(fusion_output)
        return output


# Initialize fusion model
fusion_model = FusionModel()

# Compile fusion model
fusion_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare input data
hate_labels = np.ones((len(bert_features_folder1), 1))  # Assuming labels for hate samples are 1
non_hate_labels = np.zeros((len(bert_features_folder2), 1))  # Assuming labels for non-hate samples are 0
labels = np.concatenate([hate_labels, non_hate_labels], axis=0)
bert_features = np.concatenate([bert_features_folder1, bert_features_folder2], axis=0)
mfcc_features = np.concatenate([mfcc_features_folder1_np, mfcc_features_folder2_np], axis=0)
video_features = np.concatenate([hate_video_features_np, non_hate_video_features_np], axis=0)

# Train fusion model
fusion_model.fit([bert_features, mfcc_features, video_features], labels, epochs=10, batch_size=32)

# Evaluate fusion model
loss, accuracy = fusion_model.evaluate([bert_features, mfcc_features, video_features], labels)
print("Accuracy:", accuracy)



NameError: name 'bert_features_folder1' is not defined